## Импорт и определение библиотек

In [2]:
!pip install langchain langchain_experimental langchain-openai openai tiktoken huggingface_hub langchain-huggingface -q

In [4]:
import os
from getpass import getpass
import warnings
warnings.filterwarnings('ignore')
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [5]:
# для использования ключа из LLM_bot
!wget https://raw.githubusercontent.com/a-milenkin/LLM_practical_course/main/notebooks/utils.py -q

## Выбор модели

In [53]:
# Если используете ключ из курса по LLM, запустите эту ячейку
from utils import ChatOpenAI

# Считываем ключ из файла
with open('../utils.key', 'r') as file:
    for line in file:
        if line.startswith('OPENAI_API_KEY'):
            # Извлекаем ключ, убирая лишние пробелы и символы
            course_api_key = line.split('=')[1].strip()
            break

#print(course_api_key)
# инициализируем языковую модель
llm = ChatOpenAI(temperature=0.0, course_api_key=course_api_key)

In [44]:
## Если не хотите платить денежков, то запустите эту ячейку
#from langchain.chains import LLMChain
#from langchain_huggingface import HuggingFaceEndpoint
#from langchain.embeddings import HuggingFaceEmbeddings

# Считываем ключ из файла
#with open('../utils.key', 'r') as file:
#    for line in file:
#        if line.startswith('HUGGINGFACEHUB_API_TOKEN'):
            # Извлекаем ключ, убирая лишние пробелы и символы
#            course_api_key = line.split('=')[1].strip()
#            break

# Создаем объект языкового модели
#llm = HuggingFaceEndpoint(
#      repo_id="mistralai/Mistral-7B-Instruct-v0.3",
#      task="text-generation",  # Вид задачи, в нашем случае - генерация текста
#    )

## Задаём общие параметры

import requests
import re

def search_price(ingredient):
    """
    Функция для поиска цен на алкогольные ингредиенты в нескольких интернет-магазинах.
    """
    stores = [
        {
            "name": "Winelab",
            "url": lambda ing: f"https://www.winelab.ru/search?text={ing}",
            "cookies": {"age-confirmed": "1"},
            "title_pattern": r'<a class="product-card__link" .*? title="(.*?)"',
            "price_pattern": r'<div class="price__main-value">(.*?)</div>',
            "link_pattern": r'<a class="product-card__link" href="(.*?)"',
            "link_prefix": "https://www.winelab.ru"
        },
        {
            "name": "SimpleWine",
            "url": lambda ing: f"https://simplewine.ru/catalog/?q={ing}",
            "cookies": {"age-confirm": "1"},
            "title_pattern": r'<a class="product-card__title-link" .*?>(.*?)</a>',
            "price_pattern": r'<div class="product-card__price-current">(.*?)</div>',
            "link_pattern": r'<a class="product-card__title-link" href="(.*?)"',
            "link_prefix": "https://simplewine.ru"
        }
    ]

    results = []
    
    for store in stores:
        try:
            response = requests.get(store["url"](ingredient), timeout=5)
            if response.status_code == 200:
                # Извлечение данных о товарах
                titles = re.findall(store["title_pattern"], response.text)
                prices = re.findall(store["price_pattern"], response.text)
                links = re.findall(store["link_pattern"], response.text)
                
                for title, price, link in zip(titles[:3], prices[:3], links[:3]):
                    full_link = f"{store['link_prefix']}{link}"
                    results.append(f"- {title.strip()} {price.strip()} рублей {full_link} ({store['name']})")
                
                if results:
                    break  # Если найдены результаты, завершаем поиск
            else:
                results.append(f"Ошибка доступа к {store['name']} для {ingredient}: статус {response.status_code}")
        except Exception as e:
            results.append(f"Ошибка при выполнении поиска в {store['name']} для {ingredient}: {str(e)}")

    if not results:
        results.append(f"Данные о наличии в продаже {ingredient} не найдены.")

    return "\n".join(results)

# Пример использования
ingredient_name = "водка"
print(search_price(ingredient_name))

In [81]:
import requests
import re

def search_price_with_specific_cookies(ingredient):
    """
    Функция для поиска цен на алкогольные ингредиенты с индивидуальными куки для каждого магазина.
    """
    stores = [
        {
            "name": "Winelab",
            "url": lambda ing: f"https://www.winelab.ru/search?text={ing}",
            "cookies": {"age-confirmed": "1"},
            "title_pattern": r'<a class="product-card__link" .*? title="(.*?)"',
            "price_pattern": r'<div class="price__main-value">(.*?)</div>',
            "link_pattern": r'<a class="product-card__link" href="(.*?)"',
            "link_prefix": "https://www.winelab.ru"
        },
        {
            "name": "SimpleWine",
            "url": lambda ing: f"https://simplewine.ru/catalog/?q={ing}",
            "cookies": {"age-confirm": "1"},
            "title_pattern": r'<a class="product-card__title-link" .*?>(.*?)</a>',
            "price_pattern": r'<div class="product-card__price-current">(.*?)</div>',
            "link_pattern": r'<a class="product-card__title-link" href="(.*?)"',
            "link_prefix": "https://simplewine.ru"
        }
    ]

    results = []

    for store in stores:
        try:
            headers = {
                "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:133.0) Gecko/20100101 Firefox/133.0"
            }
            response = requests.get(store["url"](ingredient), headers=headers, cookies=store["cookies"], timeout=5)
            if response.status_code == 200:
                # Извлечение данных о товарах
                titles = re.findall(store["title_pattern"], response.text)
                prices = re.findall(store["price_pattern"], response.text)
                links = re.findall(store["link_pattern"], response.text)

                for title, price, link in zip(titles[:3], prices[:3], links[:3]):
                    full_link = f"{store['link_prefix']}{link}"
                    results.append(f"- {title.strip()} {price.strip()} рублей {full_link} ({store['name']})")

                if results:
                    break  # Если найдены результаты, завершаем поиск
            else:
                results.append(f"Ошибка доступа к {store['name']} для {ingredient}: статус {response.status_code}")
        except Exception as e:
            results.append(f"Ошибка при выполнении поиска в {store['name']} для {ingredient}: {str(e)}")

    if not results:
        results.append(f"Данные о наличии в продаже {ingredient} не найдены.")

    return "\n".join(results)

# Пример использования
ingredient_name = "водка"
print(search_price_with_specific_cookies(ingredient_name))


Данные о наличии в продаже водка не найдены.


In [58]:
# Запрашиваем у пользователя ввод ингредиентов
user_input = input("Введите ингредиенты, разделенные запятыми: ")

# Преобразуем введенную строку в список, удаляя лишние пробелы
ingredients = [ingredient.strip() for ingredient in user_input.split(",")]

# Форматируем строку с использованием join для соединения элементов массива
ingredients_str = ", ".join(ingredients)

## Получаем рецепт

In [59]:
# Задаем системный промт
system_prompt = "Ты опытный бармен, специализирующийся на алкогольных коктейлях."

# Задаем промт пользователя
user_prompt = (f"Напиши, какие коктейли ты можешь изготовить из представленных ниже ингредиентов. "
               f"Также укажи, как ещё составляющие входят в коктейли и в какой пропорции изготавливаются. "
               f"Дополнительно укажи, какие ингредиенты нужно добавить, чтобы получились другие коктейли. "
               f"\nИнгредиенты: {ingredients_str}")

# Выводим сформированный промт
print(user_prompt)

Напиши, какие коктейли ты можешь изготовить из представленных ниже ингредиентов. Также укажи, как ещё составляющие входят в коктейли и в какой пропорции изготавливаются. Дополнительно укажи, какие ингредиенты нужно добавить, чтобы получились другие коктейли. 
Ингредиенты: бейлиз, эспрессо


In [60]:
messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
# Получаем ответ от модели
response = llm(messages).content

# Выводим ответ
print("Рецепт коктейля:")
print(response)

Рецепт коктейля:
Из представленных ингредиентов - бейлиз и эспрессо, можно приготовить несколько вкусных коктейлей:

1. **Ирландский кофе (Irish Coffee):**
   - 50 мл бейлиз
   - 150 мл горячего эспрессо
   - 2 чайные ложки коричневого сахара
   - Взбитые сливки для украшения

   Способ приготовления: В предварительно разогретый стакан для ирландского кофе добавьте коричневый сахар, затем налейте бейлиз и горячее эспрессо. Перемешайте, чтобы сахар растворился. Украсьте взбитыми сливками.

2. **Бейлис мартини (Baileys Martini):**
   - 50 мл бейлиз
   - 50 мл водки
   - Лед

   Способ приготовления: В шейкер с льдом добавьте бейлиз и водку. Хорошо взболтайте и процедите в охлажденный коктейльный бокал.

Для приготовления других коктейлей можно добавить следующие ингредиенты:

- Для коктейля **"Бейлис и крем" (Baileys and Cream)** добавьте 50 мл бейлиза и 50 мл сливок в стакан с льдом, аккуратно перемешайте.
- Для коктейля **"Эспрессо мартини" (Espresso Martini)** добавьте 50 мл водки, 25

In [63]:
# Функция выбора стиля ответа

def choose_style():
    styles = {
        "1": "произведения Лавкрафта",
        "2": "гопническо-быдляцкий жаргон",
        "3": "камасутра",
        "4": "Стиль барного меню"  # Стиль по умолчанию
    }
    print("Доступные стили ответа:")
    for key, value in styles.items():
        print(f"{key}: {value}")
    chosen_style = input("Выберите номер стиля (по умолчанию 4): ") or "4"
    return styles.get(chosen_style, "style4")

style = choose_style()
prompt = PromptTemplate(input_variables=['output_text', 'style'],
                        template='''Перепиши этот текст в заданном стиле: {output_text}\nСтиль: {style}.\nРезультат:''')
style_changer_chain = prompt | llm

styled_response = style_changer_chain.invoke({'output_text': response, 'style': style}).content
print("\nОтвет в выбранном стиле:")
print(styled_response)

Доступные стили ответа:
1: произведения Лавкрафта
2: гопническо-быдляцкий жаргон
3: камасутра
4: Стиль барного меню

Ответ в выбранном стиле:
Из представленных ингредиентов - бейлиз и эспрессо, можно создать несколько соблазнительных коктейлей:

1. **Ирландский кофе (Irish Coffee):**
   - 50 мл бейлиза
   - 150 мл горячего эспрессо
   - 2 чайные ложки коричневого сахара
   - Взбитые сливки для украшения

   Способ приготовления: В предварительно разогретый стакан для ирландского кофе добавьте коричневый сахар, затем налейте бейлиз и горячее эспрессо. Перемешайте, чтобы сахар растворился. Украсьте взбитыми сливками.

2. **Бейлис мартини (Baileys Martini):**
   - 50 мл бейлиза
   - 50 мл водки
   - Лед

   Способ приготовления: В шейкер с льдом добавьте бейлиз и водку. Хорошо взболтайте и процедите в охлажденный коктейльный бокал.

Для приготовления других коктейлей можно добавить следующие ингредиенты:

- Для коктейля **"Бейлис и крем" (Baileys and Cream)** добавьте 50 мл бейлиза и 50 м

In [68]:
# Разбор коктейля и ингредиентов для поиска цен
pattern = r"\*\*(.*?)\*\*:.*?((?:\n\s*-\s*.*?\n)+)"
match = re.search(pattern, response, re.DOTALL)
if match:
    cocktail_name = match.group(1)
    ingredients_section = match.group(2)
    print(f"\nКоктейль: {cocktail_name}")
    print(f"Ингредиенты:")
    print(ingredients_section)
    
    # Поиск цен для каждого ингредиента
    ingredient_lines = ingredients_section.strip().split("\n")
    for line in ingredient_lines:
        ingredient_match = re.search(r"-\s*\d+\s*мл\s*([^\n]+)", line)
        if ingredient_match:
            ingredient_name = ingredient_match.group(1)
            print(f"\nИщем цену для: {ingredient_name}")
            prices = search_price(ingredient_name)
            print(prices)
        else:
            print(f"\nНе удалось извлечь название ингредиента из строки: {line}")
else:
    print("Не удалось найти информацию о коктейле и ингредиентах в ответе модели.")

Не удалось найти информацию о коктейле и ингредиентах в ответе модели.


## Hugging Face

In [48]:
# Если выбрали модель с HuggingFace
# Объединяем системный и пользовательский промты в одну строку
full_prompt = f"{system_prompt}\n{user_prompt}"

# Получаем ответ от модели
response = llm(full_prompt)

# Выводим ответ
print(response)

, лимонный сок, сироп симментал, кубинская мед (демерара), лимон, сахар, сироп агаве, эспарто, айс.

1. Мейпл Мартини
Добавить: 60 мл джина, 30 мл сиропа симментал, 10 мл эспарто, 15 мл лимонного сока, 3 кусочки айса. Шедовить в коктейльный shaker, подмешать. Вылить в коктейльный стакан и гарнир длинным лимонным ломтом.

2. Тэкилка Дэнцерес
Добавить: 60 мл текилы, 30 мл лимонного сока, 20 мл сиропа агаве, 3 кусочки айса. Шедовить в коктейльный shaker, подмешать. Вылить в коктейльный стакан и гарнир листом из детского шоколада.

3. Кубинский дизайнер
Добавить: 60 мл джина, 30 мл лимонного сока, 20 мл кубинской мед (демерара), 3 кусочки айса. Шедовить в коктейльный shaker, подмешать. Вылить в коктейльный стакан и гарнир резко поджаренным длинным лимонным ломтом.

4. Космополитан
Добавить: 60 мл варенья курага, 30 мл текилы, 30 мл лимонного сока, 15 мл сиропа симментал, 3 кусочки айса. Шедовить в коктейльный shaker, подмешать. Вылить в коктейльный стакан и гарнир резко поджаренным ломтом 

In [49]:
# создадим шаблон и промпт
template = '''Перепиши этот текcт в заданном стиле: {output_text}
Стиль: {style}.
Результат:'''

prompt = PromptTemplate(input_variables=['output_text', 'style'], template=template)

style_changer_chain = prompt | llm # И ВСЁ!

In [ ]:
# Если используешь модель от HuggingFace
text = response
style = 'произведения Чехова'

answer = style_changer_chain.invoke({'output_text': text, 'style': style})#.content
print(answer)

In [ ]:
# Если используешь модель от OpenAI
text = response
style = 'произведения Лавкрафта'

answer = style_changer_chain.invoke({'output_text': text, 'style': style}).content
print(answer)